In [1]:
import os
import sys
import numpy as np
import pandas as pd
import json
import random

In [2]:
train_file = '/data/sulixin/research/contest/hotpot/hotpot_train_v1.1.json'
dev_full_file = '/data/sulixin/research/contest/hotpot/hotpot_dev_fullwiki_v1.json'
dev_dis_file = '/data/sulixin/research/contest/hotpot/hotpot_dev_distractor_v1.json'

In [3]:
train = json.load(open(train_file))
dev_full = json.load(open(dev_full_file))
dev_dis = json.load(open(dev_dis_file))

In [4]:
dev_dis[0]

{'_id': '5a8b57f25542995d1e6f1371',
 'answer': 'yes',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'supporting_facts': [['Scott Derrickson', 0], ['Ed Wood', 0]],
 'context': [['Ed Wood (film)',
   ['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
    " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.",
    ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.']],
  ['Scott Derrickson',
   ['Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.',
    ' He lives in Los Angeles, California.',
    ' He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinema

In [5]:
def word_len(text):
    return len(text.split())

def avg(li): 
    if len(li)==0:
        return 0
    return sum(li) * 1.0 / len(li)

def sent_cnt(contexts):
    return avg( [len(p[1]) for p in contexts] )

def passage_len(passage):
    return sum([word_len(sent) for sent in passage[1]])

def title_len(contexts):
    if len(contexts) == 0:
        return 0
    return avg([len(p[0].split()) for p in contexts if p])

def avg_p_len(contexts):
    return avg([passage_len(p) for p in contexts])
        
def my_stat(data):
    df = pd.DataFrame(data)
    df['问题长度'] = df['question'].apply(word_len)
    df['答案长度'] = df['answer'].apply(word_len)
    df['文本数量'] = df['context'].apply(len)
    df['文本长度'] = df['context'].apply(avg_p_len)
    df['证据文档数'] = df['supporting_facts'].apply(len)
    df['句子数量'] = df['context'].apply(sent_cnt)
    df['标题长度'] = df['context'].apply(title_len)
    return df.describe()

In [6]:
my_stat(train)

,问题长度,答案长度,文本数量,文本长度,证据文档数,句子数量,标题长度
count,90447.000000,90447.000000,90447.000000,90447.000000,90447.000000,90447.000000,90447.000000
mean,17.818402,2.226287,9.946897,88.941357,2.384645,4.111189,3.083111
std,9.513881,1.809021,0.590802,24.615028,0.672890,1.095305,0.766290
min,3.000000,1.000000,2.000000,14.500000,2.000000,1.000000,1.000000
25%,12.000000,1.000000,10.000000,72.300000,2.000000,3.400000,2.600000
50%,15.000000,2.000000,10.000000,86.800000,2.000000,4.000000,3.000000
75%,21.000000,3.000000,10.000000,102.900000,3.000000,4.700000,3.500000
max,108.000000,89.000000,10.000000,279.200000,12.000000,14.400000,8.500000


In [7]:
my_stat(dev_dis)

,问题长度,答案长度,文本数量,文本长度,证据文档数,句子数量,标题长度
count,7405.000000,7405.000000,7405.000000,7405.000000,7405.000000,7405.000000,7405.00000
mean,15.721404,2.461985,9.952735,89.980630,2.431465,4.153713,3.07216
std,5.522390,1.817681,0.558498,24.578971,0.711531,1.105089,0.76605
min,6.000000,1.000000,2.000000,18.700000,2.000000,1.000000,1.00000
25%,12.000000,1.000000,10.000000,73.500000,2.000000,3.400000,2.50000
50%,15.000000,2.000000,10.000000,87.600000,2.000000,4.000000,3.00000
75%,19.000000,3.000000,10.000000,103.900000,3.000000,4.700000,3.50000
max,46.000000,29.000000,10.000000,258.700000,8.000000,14.700000,7.70000


In [8]:
my_stat(dev_full)

,问题长度,答案长度,文本数量,文本长度,证据文档数,句子数量,标题长度
count,7405.000000,7405.000000,7405.000000,7405.000000,7405.000000,7405.000000,7405.000000
mean,15.721404,2.461985,9.944902,92.361568,2.431465,4.262854,3.123751
std,5.522390,1.817681,0.644505,26.411746,0.711531,1.185284,0.813969
min,6.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,12.000000,1.000000,10.000000,74.900000,2.000000,3.500000,2.500000
50%,15.000000,2.000000,10.000000,89.900000,2.000000,4.100000,3.000000
75%,19.000000,3.000000,10.000000,107.100000,3.000000,4.900000,3.600000
max,46.000000,29.000000,10.000000,266.000000,8.000000,14.800000,8.000000
